In [263]:
from preprocess import *
from df_edit import *

In [ ]:
!pip install googletrans==4.0.0-rc1

In [1]:
from googletrans import Translator

translator = Translator()

In [13]:
print(translator.translate('호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 명칭을 통일하였다.', src='ko', dest='en'))

Translated(src=ko, dest=en, text=Honam-based, the right futures, alternative and Democratic Peace, were worthy of the end of the twist, and unified the name (tentative)., pronunciation=None, extra_data="{'confiden...")


In [2]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-3c150f134808>:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


#### https://me2.kr/0j4cp 보고 따라하고 Google이 나오면 됩니다.

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome import service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

options = Options()
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems

s=Service('/opt/ml/chromedriver')
driver = webdriver.Chrome(options=options, service=s)
driver.get('http://google.com/')
myPageTitle = driver.title
print(myPageTitle)
driver.quit()

Google


In [285]:
train_df = pd.read_csv('/opt/ml/dataset/train/train.csv')
train_df_2 = better_df(train_df)
train_df_2 = Preprocess(train_df_2)

In [286]:
# index 잘못 되어 있는 지 확인
only_BMP_pattern = re.compile("["
    u"\U00010000-\U0010FFFF"  #BMP characters 이외
                       "]+", flags=re.UNICODE)
for id in train_df_2['id']:
    # while loop 돌면서 문장안에 BMP 문자 이외의 단어를 발견 할 때마다 space * 단어길이로 치환함.
    # 한 문장 안의 여러 특수 문자 있을 때에도 성공적으로 제거하며, obj,sbj index 건드리지 않음.
    while only_BMP_pattern.search(train_df_2['sentence'].loc[id]) != None:
        start_index = only_BMP_pattern.search(train_df_2['sentence'].loc[id]).start()
        end_index = only_BMP_pattern.search(train_df_2['sentence'].loc[id]).end()
        space_len = end_index-start_index
        train_df_2['sentence'].loc[id] = train_df_2['sentence'].loc[id][:start_index] + ' ' * space_len + train_df_2['sentence'].loc[id][end_index:]

In [291]:
# index 잘못 되어 있는 지 확인
for i in range(len(train_df_2)):
    if train_df_2['sentence'].iloc[i][train_df_2['subj_start'].iloc[i]:train_df_2['subj_end'].iloc[i]+1] != train_df_2['subj_word'].iloc[i]:
        print(train_df_2['sentence'].iloc[i])
        print(train_df_2['subj_word'].iloc[i])

In [294]:
train_df_2['sentence'].loc[3926]

'혜령군 이정 또는 이지(惠寧君 李  또는 李祉, 1407년 ~ 1440년)은 조선의 왕족으로 태종의 넷째 서자이다.'

In [295]:
train_df_2['sentence'].loc[31939]

'마가다(   )는 기원전 684년부터 기원전 321년까지 북인도의 비하르 남부를 중심으로 존재하던 십육대국 국가이다.'

In [296]:
# augmentation data 용 id : index dictionary제작
sen_dict = train_df_2.to_dict()
sub_ent_dict = {}
obj_ent_dict = {}

for idx in sen_dict['sentence'].keys():
    sub_ent_dict[idx]=(sen_dict['subj_start'][idx],sen_dict['subj_end'][idx])
    obj_ent_dict[idx]=(sen_dict['obj_start'][idx],sen_dict['obj_end'][idx])

In [297]:
# encode dict에 encode된 문장 저장
sub_code = "VKV" # entity가 사라지는 것을 방지
obj_code = "VJV"
encode_dict={}
for idx in sen_dict['sentence'].keys():
    sub_start = sub_ent_dict[idx][0]
    sub_end = sub_ent_dict[idx][1]
    obj_start = obj_ent_dict[idx][0]
    obj_end = obj_ent_dict[idx][1]
    if sub_start<obj_start:
        encode_dict[idx]=(sen_dict['sentence'][idx][:sub_start]+sub_code+sen_dict['sentence'][idx][sub_end+1:obj_start]+obj_code+sen_dict['sentence'][idx][obj_end+1:])
    else :
        encode_dict[idx]=(sen_dict['sentence'][idx][:obj_start]+obj_code+sen_dict['sentence'][idx][obj_end+1:sub_start]+sub_code+sen_dict['sentence'][idx][sub_end+1:])

In [298]:
# 인코딩 확인
len(encode_dict)

32423

In [299]:
# 200자가 넘길 때마다 처음 만나는 space에서 ... 삽입
for key,item in encode_dict.items():
    if len(item)>200:
        quotioent = len(item) // 200
        new_line_idx_list=[]
        for i in range(quotioent):
            new_line_idx = 200*(i+1)
            while new_line_idx < len(item) and item[new_line_idx] != ' ':
                new_line_idx += 1
            new_line_idx_list.append(new_line_idx)
        for n_idx in new_line_idx_list:
            item = item[:n_idx]+'...'+item[n_idx+1:]
        encode_dict[key]=item

# 왜 이 짓을 하는가? 파파고는 한 문장당 200자만 번역이 가능하다 ..
        

In [300]:
len(encode_dict.keys())

32423

In [301]:
# 분업용 index 나누는 함수
split_size = len(encode_dict.keys())//6
split_key = []
for i in range(6):
    if i != 5:
        split_key.append(list(encode_dict.keys())[i*split_size:(i+1)*split_size])
    else:
        split_key.append(list(encode_dict.keys())[i*split_size:])

In [302]:
total_size=0
for i in range(6):
    print(len(split_key[i]))
    total_size += len(split_key[i])
print(total_size)

5403
5403
5403
5403
5403
5408
32423


In [303]:
chk = 0
for key in split_key[0]: # 277,3548 번 제거된 것 확인
    if key != chk:
        print(key)
        chk +=1
    chk +=1

278
3548


In [304]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import numpy as np
from tqdm.auto import tqdm
from datetime import datetime
import pickle
import itertools
# 크롬창(웹드라이버) 열기
options = Options()
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems
s=Service('/opt/ml/chromedriver')
driver = webdriver.Chrome(options=options, service=s)

    



In [163]:
import re
# 이전에 만든거 삭제 X
data = '안녕하세요 반갑습니다🐶'
for i in range(6):
        for key in split_key[i]:
                data = encode_dict[key]
                only_BMP_pattern = re.compile("["
                        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                                        "]+", flags=re.UNICODE)
                if only_BMP_pattern.findall(data) != []:
                        print(key)            
                        print(only_BMP_pattern.findall(data))# BMP characters만

3926
['𥘺']
31939
['𑀫𑀕𑀥']


In [305]:
import re

# data = '안녕하세요 반갑습니다🐶'
# 제대로 삭제되었는지 확인
for i in range(6):
        for key in split_key[i]:
                data = encode_dict[key]
                only_BMP_pattern = re.compile("["
                        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                                        "]+", flags=re.UNICODE)
                if only_BMP_pattern.findall(data) != []:
                        print(key)            
                        print(only_BMP_pattern.findall(data))# BMP characters만

In [307]:
# 파파고 페이지 접속

target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))


i = 0 # 여기에 자신이 배정받은 번호 입력
start_index = split_key[i][0]
last_index = split_key[i][-1]
kor_to_eng_dict={}

for key in split_key[i]:
    data = encode_dict[key]
    # 입력창에 검색어 입력
    backtrans=''
    # element=WebDriverWait(driver, 10).until(target_present)
    driver.get('https://papago.naver.com/?sk=ko&tk=en')
    time.sleep(0.1)
    driver.find_element(By.XPATH,value='//*[@id="sourceEditArea"]/label').send_keys(data)
    driver.find_element(By.XPATH,value='//*[@id="btnTranslate"]').click()
    time.sleep(1.5)
    backtrans = WebDriverWait(driver, 20).until(target_present).text
    # backtrans = driver.find_element(By.XPATH,value='//*[@id="txtTarget"]').text
    kor_to_eng_dict[key]=backtrans

with open(f'kor_eng_{start_index}_to_{last_index}.pickle', 'wb') as f:
    pickle.dump(kor_to_eng_dict, f, pickle.HIGHEST_PROTOCOL)

In [321]:
for idx, (key,data) in enumerate(kor_to_eng_dict.items()):
    if data == '':
        print (key)

79
750
997
1123
1126
1183
1239
1261
1374
1494
1601
1631
1639
1647
1701
1714
1742
1764
1819
1849
1866
1901
1913
1915
1920
2007
2019
2036
2045
2061
2089
2151
2200
2216
2235
2244
2279
2292
2294
2308
2309
2332
2345
2367
2400
2405
2412
2436
2440
2454
2477
2484
2486
2503
2520
2536
2547
2602
2605
2626
2639
2644
2688
2694
2697
2738
2749
2750
2759
2772
2788
2823
2851
2854
2864
2876
2890
2896
2904
2905
2910
2925
2935
2958
2962
3003
3034
3058
3088
3125
3141
3183
3191
3193
3197
3220
3223
3268
3269
3288
3308
3316
3340
3342
3355
3364
3371
3400
3421
3447
3456
3481
3502
3513
3522
3537
3539
3554
3560
3562
3597
3656
3679
3722
3753
3767
3778
3782
3849
3850
3857
3866
3902
3912
3929
3975
4006
4020
4023
4027
4033
4061
4062
4097
4101
4143
4144
4159
4161
4174
4192
4223
4243
4256
4263
4289
4308
4312
4316
4320
4321
4329
4330
4334
4339
4347
4349
4361
4382
4400
4410
4417
4419
4429
4441
4489
4495
4513
4520
4525
4538
4549
4562
4564
4577
4589
4591
4594
4612
4670
4700
4702
4722
4734
4738
4742
4747
4756
4766
4858
4866

In [320]:
from selenium.webdriver.common.keys import Keys

target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
eng_to_kor_dict={}
i=0
for idx,(key,data) in enumerate(kor_to_eng_dict.items()):
    # 입력창에 검색어 입력
    if data != '':
        backtrans=''
        # element=WebDriverWait(driver, 10).until(target_present)
        driver.get('https://papago.naver.com/?sk=en&tk=ko')
        time.sleep(0.1)
        driver.find_element(By.XPATH,value='//*[@id="sourceEditArea"]/label').send_keys(data)
        driver.find_element(By.XPATH,value='//*[@id="btnTranslate"]').send_keys(Keys.ENTER)
        time.sleep(1.5)
        backtrans = WebDriverWait(driver, 20).until(target_present).text
        # backtrans = driver.find_element(By.XPATH,value='//*[@id="txtTarget"]').text
        eng_to_kor_dict[key]=backtrans
        if (idx+1)%500 == 0 :
            start_index = idx - 499
            end_index = idx
            with open(f'eng_kor_{start_index}_to_{last_index}.pickle', 'wb') as f:
                pickle.dump(eng_to_kor_dict, f, pickle.HIGHEST_PROTOCOL)
            eng_to_kor_dict ={}
driver.close()

KeyboardInterrupt: 

In [51]:
eng_to_kor_dict

{0: '어떤 것은 VJV에 의해 쓰여졌고 VKV의 1969년 앨범 Abbey Road에 포함되어 있다.',
 1: '호남을 기반으로 하는 바른정당과 VJV, VKV가 우여곡절 끝에 합당해 민생당(가칭)으로 거듭난다.',
 2: "K리그2 1위인 VKV는 26일 VJV의 시청자 유치와 마케팅 실적을 인정받아 '풀스타디움상'과 '플러스스타디움상'을 수상했다.",
 3: '주식회사 VKV(대표 VJV)는 COVID-19 바이러스로 몸살을 앓고 있는 대구광역시에 행복상자를 전달했다고 10일 밝혔다.',
 4: '그가 VJV 프로야구 드래프트 1순위로 VKV에 입단했을 때, 그의 유니폼 번호는 8번이었다.',
 5: ': 유엔, 유럽의회, VKV(VJV), 국제이주기구, 세계보건기구(WHO), 지중해연합, 이슬람협력기구, 유럽안보협력기구, 국제통화기금, 세계무역기구.',
 6: '이에 따라 나폴리와 계약을 연장한 마라도나는 VJV팀을 UEFA컵 정상에 올려놓았고 이듬해 유럽 챔피언 VKV를 꺾고 다시 한 번 세리에A 정상에 올랐다.',
 7: 'VKV (음력 3월 19일) - 2009년 11월 4일 서울에서 태어난 대한민국 기업인으로 두산그룹 회장 및 KBO 총재를 역임하였다.',
 8: '중국군에 대해 너무 약해진 국민당은 수도를 VJV로 옮기기로 결정하고 12월부터 남아 있는 VKV 병력과 국유재산을 차례로 대만으로 이전하기 시작했고, 12월에는 중앙정부가 타이베이시를 옮겨 중화민국의 새 수도가 되었다.',
 9: '특히 김동연 전 부총리, VKV 의원, 김종민 오세 최운열 김정우 권칠승 맹성규 의원 등 8명의 VJV 의원이 영상 메시지를 보냈다.'}

In [52]:
aug_dict ={}
for i in range(1):
    with open(f'eng_kor_{i*(last_index-start_index)}_to_{(i+1)*(last_index-start_index)}.pickle','rb') as f:
        data = pickle.load(f)
        aug_dict.update(data)
aug_dict

{0: '어떤 것은 VJV에 의해 쓰여졌고 VKV의 1969년 앨범 Abbey Road에 포함되어 있다.',
 1: '호남을 기반으로 하는 바른정당과 VJV, VKV가 우여곡절 끝에 합당해 민생당(가칭)으로 거듭난다.',
 2: "K리그2 1위인 VKV는 26일 VJV의 시청자 유치와 마케팅 실적을 인정받아 '풀스타디움상'과 '플러스스타디움상'을 수상했다.",
 3: '주식회사 VKV(대표 VJV)는 COVID-19 바이러스로 몸살을 앓고 있는 대구광역시에 행복상자를 전달했다고 10일 밝혔다.',
 4: '그가 VJV 프로야구 드래프트 1순위로 VKV에 입단했을 때, 그의 유니폼 번호는 8번이었다.',
 5: ': 유엔, 유럽의회, VKV(VJV), 국제이주기구, 세계보건기구(WHO), 지중해연합, 이슬람협력기구, 유럽안보협력기구, 국제통화기금, 세계무역기구.',
 6: '이에 따라 나폴리와 계약을 연장한 마라도나는 VJV팀을 UEFA컵 정상에 올려놓았고 이듬해 유럽 챔피언 VKV를 꺾고 다시 한 번 세리에A 정상에 올랐다.',
 7: 'VKV (음력 3월 19일) - 2009년 11월 4일 서울에서 태어난 대한민국 기업인으로 두산그룹 회장 및 KBO 총재를 역임하였다.',
 8: '중국군에 대해 너무 약해진 국민당은 수도를 VJV로 옮기기로 결정하고 12월부터 남아 있는 VKV 병력과 국유재산을 차례로 대만으로 이전하기 시작했고, 12월에는 중앙정부가 타이베이시를 옮겨 중화민국의 새 수도가 되었다.',
 9: '특히 김동연 전 부총리, VKV 의원, 김종민 오세 최운열 김정우 권칠승 맹성규 의원 등 8명의 VJV 의원이 영상 메시지를 보냈다.'}

In [54]:
# 인코딩 과정에서 소문자가 나올 수 있어서 고려함
score = 0
aug_subj_idx_dict = {}
aug_obj_idx_dict = {}

for idx,item in aug_dict.items():
    if item != '': # 번역이 안된 경우 제외
        if sub_code in item or sub_code.lower() in item:
            if obj_code in item or obj_code.lower() in item:
                score += 1
                
                sub_start_idx = item.find(sub_code)
                if sub_start_idx == -1:
                    sub_start_idx = item.find(sub_code.lower())
                obj_start_idx = item.find(obj_code)
                if obj_start_idx == -1:
                    obj_start_idx = item.find(obj_code.lower())

                if sub_start_idx < obj_start_idx:
                    item=item.replace(sub_code,sen_dict['subj_word'][idx])
                    item=item.replace(sub_code.lower(),sen_dict['subj_word'][idx])
                    aug_subj_idx_dict[idx]=((sub_start_idx,sub_start_idx+len(sen_dict['subj_word'][idx])-1))

                    obj_start_idx = item.find(obj_code)
                    if obj_start_idx == -1:
                        obj_start_idx = item.find(obj_code.lower())

                    item=item.replace(obj_code,sen_dict['obj_word'][idx])
                    item=item.replace(obj_code.lower(),sen_dict['obj_word'][idx])
                    aug_obj_idx_dict[idx]=((obj_start_idx,obj_start_idx+len(sen_dict['obj_word'][idx])-1))

                else :
                    item=item.replace(obj_code,sen_dict['obj_word'][idx])
                    item=item.replace(obj_code.lower(),sen_dict['obj_word'][idx])
                    aug_obj_idx_dict[idx]=((obj_start_idx,obj_start_idx+len(sen_dict['obj_word'][idx])-1))

                    sub_start_idx = item.find(sub_code)
                    if sub_start_idx == -1:
                        sub_start_idx = item.find(sub_code.lower())

                    item=item.replace(sub_code,sen_dict['subj_word'][idx])
                    item=item.replace(sub_code.lower(),sen_dict['subj_word'][idx])
                    aug_subj_idx_dict[idx]=((sub_start_idx,sub_start_idx+len(sen_dict['subj_word'][idx])-1))

                aug_dict[idx]=item
print("restoration percent : ", score/len(aug_dict))
aug_subj_idx_dict

restoration percent :  0.9


{0: (22, 24),
 1: (24, 28),
 2: (9, 12),
 3: (5, 9),
 4: (23, 31),
 5: (12, 21),
 6: (59, 63),
 8: (52, 55),
 9: (14, 16)}

In [55]:
for idx,item in aug_dict.items():
    if idx in aug_subj_idx_dict.keys():
        print(aug_dict[idx][aug_subj_idx_dict[idx][0]:aug_subj_idx_dict[idx][1]+1])

비틀즈
민주평화당
광주FC
아성다이소
요미우리 자이언츠
북대서양 조약 기구
AC 밀란
중화민국
안규백


In [56]:
aug_data={}
for idx,item in aug_dict.items():
    if idx in aug_subj_idx_dict.keys():
        insert_data={'id':idx+32470,
                    'subj_type':sen_dict['subj_type'][idx],
                    'subj_word':sen_dict['subj_word'][idx],
                    'subj_start':aug_subj_idx_dict[idx][0],
                    'subj_end':aug_subj_idx_dict[idx][1],
                    'obj_type':sen_dict['obj_type'][idx],
                    'obj_word':sen_dict['obj_word'][idx],
                    'obj_start':aug_obj_idx_dict[idx][0],
                    'obj_end':aug_obj_idx_dict[idx][1],
                    'sentence':aug_dict[idx],
                    'label':sen_dict['label'][idx],
                    'label_num':sen_dict['label_num'][idx]}
        train_df_2 = train_df_2.append(insert_data,ignore_index=True)

In [57]:
train_df_2.tail(10) #추가됬는지 확인
# 약간 문제가 되는게 번역이 안되는 문장에 대해 index와 id가 안맞는 일이 발생함 조금 수정하면 될 것 같은데 문제 없는 이상 이대로 하겠음.

,id,subj_type,subj_word,subj_start,subj_end,obj_type,obj_word,obj_start,obj_end,sentence,label,label_num
32422,32469,ORG,화순군,0,2,PER,구충곤,7,9,"화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버스 승강장, 공중화장실 등 다중이용시설과 오동리 천운아파트 주변을 소독하는 방역 봉사활동을 펼쳤다고 밝혔다.",org:top_members/employees,1
32423,32470,ORG,비틀즈,22,24,PER,조지 해리슨,6,11,어떤 것은 조지 해리슨에 의해 쓰여졌고 비틀즈의 1969년 앨범 Abbey Road에 포함되어 있다.,no_relation,0
32424,32471,ORG,민주평화당,24,28,ORG,대안신당,18,21,"호남을 기반으로 하는 바른정당과 대안신당, 민주평화당가 우여곡절 끝에 합당해 민생당(가칭)으로 거듭난다.",no_relation,0
32425,32472,ORG,광주FC,9,12,ORG,한국프로축구연맹,19,26,K리그2 1위인 광주FC는 26일 한국프로축구연맹의 시청자 유치와 마케팅 실적을 인정받아 '풀스타디움상'과 '플러스스타디움상'을 수상했다.,org:member_of,20
32426,32473,ORG,아성다이소,5,9,PER,박정부,14,16,주식회사 아성다이소(대표 박정부)는 COVID-19 바이러스로 몸살을 앓고 있는 대구광역시에 행복상자를 전달했다고 10일 밝혔다.,org:top_members/employees,1
32427,32474,ORG,요미우리 자이언츠,23,31,DAT,1967,3,6,"그가 1967 프로야구 드래프트 1순위로 요미우리 자이언츠에 입단했을 때, 그의 유니폼 번호는 8번이었다.",no_relation,0
32428,32475,ORG,북대서양 조약 기구,12,21,ORG,NATO,23,26,": 유엔, 유럽의회, 북대서양 조약 기구(NATO), 국제이주기구, 세계보건기구(WHO), 지중해연합, 이슬람협력기구, 유럽안보협력기구, 국제통화기금, 세계무역기구.",org:alternate_names,5
32429,32476,ORG,AC 밀란,59,63,DAT,1989,25,28,이에 따라 나폴리와 계약을 연장한 마라도나는 1989팀을 UEFA컵 정상에 올려놓았고 이듬해 유럽 챔피언 AC 밀란를 꺾고 다시 한 번 세리에A 정상에 올랐다.,no_relation,0
32430,32478,ORG,중화민국,52,55,LOC,타이베이,24,27,"중국군에 대해 너무 약해진 국민당은 수도를 타이베이로 옮기기로 결정하고 12월부터 남아 있는 중화민국 병력과 국유재산을 차례로 대만으로 이전하기 시작했고, 12월에는 중앙정부가 타이베이시를 옮겨 중화민국의 새 수도가 되었다.",org:place_of_headquarters,7
32431,32479,PER,안규백,14,16,ORG,더불어민주당,54,59,"특히 김동연 전 부총리, 안규백 의원, 김종민 오세 최운열 김정우 권칠승 맹성규 의원 등 8명의 더불어민주당 의원이 영상 메시지를 보냈다.",per:employee_of,6
